# Sentiment Analysis using BERT

# Description
This Jupyter notebook will illustrate the use of BERT pre-trained model to classify a Twitter tweet sentiment as case study.

The target audience of this Jupyter notebook:
1. Had no experience in Deep NLP Model such as BERT. 
2. Have some basic understanding in Text Data Mining.
3. Curious on how to use and fine-tune the BERT model for simple training task.
4. And of course, looking forward to spending fun time with Python code!



### The algorithm steps in the software code in a nutshell:
1. Analyze the dataset using pandas dataframe
2. Split the labelled dataset into training dataset and validation dataset (Training 85% vs Validation 15%)
3. Data (from dataframe) tokenizing and encoding
4. Setting Pre-Trained BERT Model (We will use base-uncased model)
4. Fine-tune BERT BASE Modeul using the training dataset and store the model locally
5. Evaluate saved model using training dataset
6. Report the evaluation score (f1)

# Sources and References

1. BERT Illustration: http://jalammar.github.io/illustrated-bert/
2. Using BERT For the first time: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
3. Sample code taken from https://www.coursera.org/learn/sentiment-analysis-bert/home/welcome


## Dataset source

1. <b>Source cititation</b>

Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2 


2. <b>What to know about the dataset</b>

Dataset has 3085 lines (tweets). Each record consists of 3 columns.

- Column-1: The unique ID of the line/tweet
- Column-2: The tweet message
- Column-3: The sentiment label of the message (sad, happy, etc.)

Sample data from the dataset:

<b>611537640857411584,"@britishmuseum @SenderosP The Rosetta Stone ;)",happy</b>

- Column-1: 611537640857411584
- Column-2: "@britishmuseum @SenderosP The Rosetta Stone ;)"
- Column-3: happy


## Prerequisites
Before moving foward to the software section below, read the provided links below to have the intuition of each concept:
- Basic understanding of Language Model: https://en.wikipedia.org/wiki/Language_model
- Basic understading about Deep Learning: https://en.wikipedia.org/wiki/Deep_learning
- Basic understanding about BERT pre-trained models and fine-tune process: https://arxiv.org/abs/1810.04805
- Basic knowledge of Python 3, PyTorch usage, and Jupyter Notebook

# Software
The algorithm is devided into section based on the steps described in the "Description" section of this notebook

### 1. Analyze the dataset using pandas dataframe

In [3]:
import pandas as pd  # https://pandas.pydata.org/

In [4]:
# Load the dataset into dataframes
# dataframe columns: [id, text, category]
df = pd.read_csv('Data/smile-annotations-final.csv', names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [5]:
# Let take a look at how the data look like
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
# Let's take a look at each label & the counting
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [7]:
# Filter out multiple label (the ones with | character) and "nocode" label
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']

In [8]:
# Let's review the dataset/label counting after removing some unwanted tweets/label
# this removal is purely for simplicity of our training model
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
# Assign integer value to the text label. The new column is called "label"
# by now dataframe structure [id, text, category, label]
possible_labels = df.category.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
df['label'] = df.category.replace(label_dict)    

In [10]:
# Let's review the new (integer) label
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


## 2. Split the labelled dataset into training dataset and validation dataset (Training 85% vs Validation 15%)

In [111]:
# We will use sklearn library to split the dataset into training and test dataset
from sklearn.model_selection import train_test_split # https://scikit-learn.org/

In [112]:
# Here we will split the dataset as follow:
# 15% as testing dataset
# 85% as training dataset
# random_state is set to 42. Popular integer random seeds are 0 and 42.
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [113]:
# Adding a new column called "data_type", the possible value of the new columns:
# 'train' for the training dataset
# or 'val' stands for validation for the testing dataset
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

# print dataframe to see the result
df

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train
...,...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1,train
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0,train
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0,train


In [114]:
# print dataframe groung by category, label and data_type
# Here we should have 85% vs 15% distribution for each category/label
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## 3. Loading Tokenizer and Encoding our Data

In [115]:
from transformers import BertTokenizer # https://huggingface.co/transformers/model_doc/bert.html
from torch.utils.data import TensorDataset # https://pytorch.org/
import torch

In [116]:
# we create our tokenizer here
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [117]:
# tokenizer will read dataframe and encode number representation 
# which will be recognized by MODEL pre-trained models
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length',     
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [118]:
# Let's take a look at our text data in a integer representation!
# Don't worry BERT model will understand these numbers as we understand the text data
encoded_data_train

{'input_ids': tensor([[  101, 16092,  3897,  ...,     0,     0,     0],
        [  101,  1030, 27034,  ...,     0,     0,     0],
        [  101,  1030, 10682,  ...,     0,     0,     0],
        ...,
        [  101, 11047,  1030,  ...,     0,     0,     0],
        [  101,  1030,  3680,  ...,     0,     0,     0],
        [  101,  1030,  2120,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [119]:
# Create the Tensor dataset using the encoded data created in the previous step
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [120]:
# Total dataset
len(dataset_train)+len(dataset_val)

1481

In [121]:
# Total dataset by label (they are matching!, so we got all our dataset covered in TensorDataset after dataset split)
df.label.count()

# Next is to feed the Tensordataset to our BERT Model

1481

## 4. Setting Pre-Trained BERT Model (We will use base-uncased model)

In [122]:
# Here, we will use a pre-tained BERT model BertForSequenceClassification
# For complete list of BERT Model available in huggingface: https://huggingface.co/models
from transformers import BertForSequenceClassification

In [123]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Creating Data Loaders

In [124]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [125]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

## Setting Up Optimiser and Scheduler

In [126]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [127]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [128]:
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [129]:
import numpy as np

In [130]:
from sklearn.metrics import f1_score

In [131]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [132]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [133]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [134]:
# Check NVIDIA CUDA availability
torch.cuda.is_available()

True

In [139]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model.to(device)


print(device)

cpu


In [140]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [141]:
from tqdm.notebook import tqdm # https://github.com/tqdm/tqdm

In [142]:
for epoch in tqdm(range(epochs)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 0
Training loss: 1.0163526117801667
Validation loss: 0.7596349375588554
F1 Score (Weighted): 0.6656119824269878



Epoch 1
Training loss: 0.7081792920827865
Validation loss: 0.6815478461129325
F1 Score (Weighted): 0.7441707586955318



Epoch 2
Training loss: 0.6147345200181007
Validation loss: 0.5934487410954067
F1 Score (Weighted): 0.7510315814595022



Epoch 3
Training loss: 0.5481138855218888
Validation loss: 0.5896298970494952
F1 Score (Weighted): 0.7510315814595022



In [143]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [144]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [145]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [146]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 169/171

Class: not-relevant
Accuracy: 11/32

Class: angry
Accuracy: 0/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 0/5

Class: surprise
Accuracy: 0/5

